In [187]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 8
year = 2022

data = get_data(day=day, year=year)

# submit(cube.sum(), part="a", day=day, year=year)

# data

In [142]:
data = """30373
25512
65332
33549
35390"""

In [195]:
A = np.array([[int(i) for i in l] for l in data.split("\n")])

visible = lambda l: [k > max([-1, *l[:i]]) for i,k in enumerate(l)]

view = lambda l: [next((j+1 for j,v in enumerate(l[i+1:]) if v >= l[i]), len(l)-i-1) for i in range(len(l)-1)] + [0]

apply4sides = lambda fn, X: [np.rot90(np.apply_along_axis(fn, 1, np.rot90(X, k=k)), k=4-k) for k in range(4)]

mprod = lambda X: np.prod(np.array(X), axis=0)

print("Part 1:", (sum(apply4sides(visible, A)) > 0).sum())
print("Part 2:", mprod(apply4sides(view, A)).max())


Part 1: 1835
Part 2: 263670


1835

array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0]])

['D', 'C']